In [1]:
#imports
import requests
import json
import pandas as pd
import numpy as np
import warnings
import random
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
import json
import pandas as pd
import numpy as np
import time

start_time = time.time()

Sensor_readings = pd.read_json('data/W512_readings.json')
Aircon_Data = pd.read_json('data/W512_aircon_status.json')
Weather_readings = pd.read_json('data/Weather_data.json')

def convert_AirconData(data):
    records = []
    
    for index, row in data.iterrows():
        # Parse FC_FullStatus_Readings if it's a string representation of a dictionary
        if isinstance(row['FC_FullStatus_Readings'], str):
            fc_readings = ast.literal_eval(row['FC_FullStatus_Readings'])
        else:
            fc_readings = row['FC_FullStatus_Readings']
        try:
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))

        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            combined_datetime = None
            formatted_datetime = None

        
        # Create a record with base information
        record = {
            'Datetime': formatted_datetime
        }
        
        # Add each FC Unit's details as separate columns
        for unit, unit_data in fc_readings.items():
            record[f'{unit}_Status'] = unit_data['Status']
            record[f'{unit}_Fan_Status'] = unit_data['Fan_Status']
            record[f'{unit}_Set_Point'] = unit_data['Set_Point']
            record[f'{unit}_Operation_Mode'] = unit_data['Operation_Mode']
        
        records.append(record)
    
    # Create DataFrame
    df = pd.DataFrame(records)
    return df


def convert_sensorReadings(data):
    records = []
    
    # List of keys to exclude from Lorawan_Readings
    include_keys_1 = ["24E124725E285123", "24E124725E331695","24E124725E331744",
                      "24E124725E332483","24E124725E290348","24E124725E331733","24E124725E286745"]#"24E124136D316361" is suppiosed to be outdoor but it is not outdoor yet
    include_keys_2 = ["Sensor_1","Sensor_3","Sensor_6"]
    
    for index, row in data.iterrows():
        # Parse Energy_Readings if it's a string representation of a dictionary
        if isinstance(row['Energy_Readings'], str):
            Energy_readings = ast.literal_eval(row['Energy_Readings'])
        else:
            Energy_readings = row['Energy_Readings']
        # Parse Lorawan_Readings if it's a string representation of a dictionary
        if isinstance(row['Lorawan_Readings'], str):
            Lorawan_Readings = ast.literal_eval(row['Lorawan_Readings'])
        else:
            Lorawan_Readings = row['Lorawan_Readings']

        try:
            # Combine the date and time columns to create a datetime object
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))
        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            formatted_datetime = None

        # Create a record with base information
        record = {
            'Datetime': formatted_datetime
        }
        
        # Add each Energy sensor's details as separate columns
        for unit, unit_data in Energy_readings.items():
            if unit not in include_keys_2:
                continue
                
            record[f'{unit}_Current'] = unit_data['Current']
            record[f'{unit}_Energy'] = unit_data['Energy']
            record[f'{unit}_Power'] = unit_data['Power']
        
        # Add each Lorawan device's details as separate columns
        for unit, unit_data in Lorawan_Readings.items():
            if unit not in include_keys_1:
                continue
            record[f'{unit}_Humidity'] = unit_data.get('humidity', None)
            record[f'{unit}_Temperature'] = unit_data.get('temperature', None)

            co2_value = unit_data.get('co2', None)
            if co2_value is not None:
                record[f'{unit}_CO2'] = co2_value

        # Append the record to the list of records
        records.append(record)
    df=pd.DataFrame(records)
    return df


def convert_weatherData(data):
    records = []
    for index, row in data.iterrows():
        # Parse Energy_Readings if it's a string representation of a dictionary
        if isinstance(row['result'], str):
            weather_results = ast.literal_eval(row['result'])
        else:
            weather_results = row['result']
            
        try:
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))
        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            formatted_datetime = None

        record = {
            'Datetime': formatted_datetime
        }  

        record['weather_status'] = weather_results['weather_status']
        record['weather_temp'] = weather_results['weather_temp']
        record['weather_humid'] = weather_results['weather_humidity']
            
        records.append(record)
    df=pd.DataFrame(records)
    return df



Aircon_data_df = convert_AirconData(Aircon_Data)
Aircon_data_df = Aircon_data_df[3194:] #after row 3192 is the aircon with status
Sensor_readings_df = convert_sensorReadings(Sensor_readings)
Sensor_readings_df = Sensor_readings_df.interpolate(method='linear')
weather_readings_df = convert_weatherData(Weather_readings)

# Merge Aircon data with Sensor readings using merge_asof
merged_df = pd.merge_asof(Aircon_data_df, Sensor_readings_df, on='Datetime', direction='nearest')

# Now, merge the Weather readings with the previous result using merge_asof
merged_df = pd.merge_asof(merged_df, weather_readings_df, on='Datetime', direction='nearest')


merged_df['total_energy'] = (
    merged_df['Sensor_1_Energy'] +
    merged_df['Sensor_3_Energy'] +
    merged_df['Sensor_6_Energy']
)

merged_df['total_power'] = (
    merged_df['Sensor_1_Power'] +
    merged_df['Sensor_3_Power'] +
    merged_df['Sensor_6_Power']
)

merged_df['total_current'] = (
    merged_df['Sensor_1_Current'] +
    merged_df['Sensor_3_Current'] +
    merged_df['Sensor_6_Current']
)

temperature_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "temperature" in col.lower()
]
humidity_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "humidity" in col.lower()
]
co2_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "co2" in col.lower()
]

merged_df['avg_temperature'] = merged_df[temperature_col].mean(axis=1)
merged_df['avg_humidity'] = merged_df[humidity_col].mean(axis=1)
merged_df['avg_co2'] = merged_df[co2_col].mean(axis=1)

dropped_col = [
    col for col in merged_df.columns
    if "24e124" in col.lower()
]
dropped_col_sensor = [
    col for col in merged_df.columns
    if "sensor" in col.lower()
]

columns_to_drop = dropped_col + dropped_col_sensor
merged_df.drop(columns=columns_to_drop, axis=1, inplace=True)
merged_df['timestamp'] = merged_df['Datetime'].apply(lambda x: int(x.timestamp()))
merged_df.drop(columns="Datetime", axis=1, inplace=True)

#########################################################################################################################

aircon_units = len([
    col for col in merged_df.columns
    if "FC_Unit_" in col and "_Status" in col and "Fan" not in col
])

def getFCData(data,row_index):
    settings = []
    for i in range(1,aircon_units + 1):
        settings.append(data[f"FC_Unit_{i}_Status"].iloc[row_index])
        settings.append(data[f"FC_Unit_{i}_Fan_Status"].iloc[row_index])
        settings.append(data[f"FC_Unit_{i}_Set_Point"].iloc[row_index])
        settings.append(data[f"FC_Unit_{i}_Operation_Mode"].iloc[row_index])

    return settings

def is_same_settings(data,curr_row_index, next_row_index):
    return True if (getFCData(data, curr_row_index) == getFCData(data, next_row_index)) else False

def is_all_off(data, curr_row_index, check_for_off):
    for i in range(1, aircon_units + 1):
        if data[f"FC_Unit_{i}_Status"].iloc[curr_row_index] == "ON":
            return not check_for_off
        
    return check_for_off        
    
aircon_status_result = pd.DataFrame()

num_of_rows = merged_df.shape[0]



for i in range(num_of_rows - 1, -1, -1):
    rows = []
    time_taken = []
    energy_consumption = []
    previous_temp = []
    previous_humi = []
    
    curr_timestamp = merged_df["timestamp"].iloc[i]
    curr_energy = merged_df["total_energy"].iloc[i]
    curr_temperature = merged_df["avg_temperature"].iloc[i]
    curr_humidity = merged_df["avg_humidity"].iloc[i]
    
    while i >= 0 and is_same_settings(merged_df, i - 1, i):
        rows.append(i - 1)
        time_taken.append(curr_timestamp - merged_df["timestamp"].iloc[i - 1])
        energy_consumption.append(round(curr_energy - merged_df["total_energy"].iloc[i - 1],2))
        previous_temp.append(round(merged_df["avg_temperature"].iloc[i - 1],2))
        previous_humi.append(round(merged_df["avg_humidity"].iloc[i - 1],2))
        
        i -= 1
        
    temp_df = pd.DataFrame({
            'timestamp': [curr_timestamp],
            'rows': [rows],
            'time_taken': [time_taken],
            'energy_consumption': [energy_consumption],
            'previous_temp': [previous_temp],
            'previous_humi': [previous_humi],
            'current_temp': [curr_temperature],
            'current_humi': [curr_humidity]
        })
    for col in merged_df.columns:
        if "FC_Unit_" in col:
            temp_df[col] = merged_df[col].iloc[i]
    
    # Concatenate temp_df to the main result DataFrame
    aircon_status_result = pd.concat([aircon_status_result, temp_df], ignore_index=True)



print("Finished")
aircon_status_result = aircon_status_result.sort_values(by=['current_temp'], ascending=False)
aircon_status_result.info()
aircon_status_result.to_csv('aircon_status2.csv', index=False)

end_time = time.time()
elapsed_time_minutes = (end_time - start_time) / 60
print(f"Processing completed in {elapsed_time_minutes:.2f} minutes")

Finished
<class 'pandas.core.frame.DataFrame'>
Index: 4066 entries, 2742 to 4048
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   timestamp                 4066 non-null   int64  
 1   rows                      4066 non-null   object 
 2   time_taken                4066 non-null   object 
 3   energy_consumption        4066 non-null   object 
 4   previous_temp             4066 non-null   object 
 5   previous_humi             4066 non-null   object 
 6   current_temp              4066 non-null   float64
 7   current_humi              4066 non-null   float64
 8   FC_Unit_1_Status          4066 non-null   object 
 9   FC_Unit_1_Fan_Status      4066 non-null   object 
 10  FC_Unit_1_Set_Point       4066 non-null   float64
 11  FC_Unit_1_Operation_Mode  4066 non-null   object 
 12  FC_Unit_2_Status          4066 non-null   object 
 13  FC_Unit_2_Fan_Status      4066 non-null   object 
 14  F

In [12]:
target_temp = 24
target_temp_range = range(20, 30)
time_factor = 0.5
energy_factor = 0.5
acceptable_range = 0.8
total_rows = aircon_status_result.shape[0]
stored_dictionary = {}


start_time = time.time()

def getRowData(row_index):
    temperature = aircon_status_result["current_temp"].iloc[row_index]
    humidity = aircon_status_result["current_humi"].iloc[row_index]
    
    return [temperature, humidity]

def getArrayData(row_index, array_index):
    time_taken = aircon_status_result["time_taken"].iloc[row_index]
    energy_consumption = aircon_status_result["energy_consumption"].iloc[row_index]
    temperature = aircon_status_result["previous_temp"].iloc[row_index]
    humidity = aircon_status_result["previous_humi"].iloc[row_index]
    
    return [temperature[array_index], humidity[array_index], time_taken[array_index], energy_consumption[array_index]]

def comparePath(best_path, current_path):    
    if best_path['factor'] > current_path['factor']:
        return True
    
    return False


for target_temp in target_temp_range:
    paths = {}
    # returns energy consumption and time taken and best path
    # every combi would have different settings
    def findBestCombi(current_row_index):
        global paths
        # Target not reached
        # ALl aircon status is OFF, not useful 
        
        curr_temperature, curr_humidity = getRowData(current_row_index)
        
        if is_all_off(aircon_status_result, current_row_index, True):
            paths[current_row_index] = {
                'energy_consumption': [], 
                'starting_temp': curr_temperature, 
                'starting_humi': curr_humidity, 
                'time_taken': [], 
                'factor': float('inf'), 
                'path': []
            }
            return paths[current_row_index]
        
        # If current_row_index is already checked before
        if current_row_index in paths:
            # Can be EMPTY or VALID PATH
            return paths[current_row_index]
        
        # Check if using this status hit the target temp and humi
        for i in range(len(aircon_status_result['rows'].iloc[current_row_index])):
            array_data = getArrayData(current_row_index, i)
    
            #the moment it finds a temperature in the "previous temp array" that is in the acceptance from the target it will be recorded down
            if (abs(array_data[0] - target_temp) < acceptable_range):
                print("target found")
                # Target Found
                # Put entry in paths
                curr_path = {
                    'energy_consumption': [array_data[3]],
                    'time_taken': [array_data[2]],
                    'factor': array_data[3] * energy_factor + array_data[2] * time_factor,
                    'starting_temp': curr_temperature,
                    'starting_humi': curr_humidity,
                    'ending_temp': array_data[0],
                    'ending_humi': array_data[1],
                    'path': [current_row_index]
                }
                paths[current_row_index] = curr_path
                return paths[current_row_index]
        
        # Start of Backtracking
        for i in range(len(aircon_status_result['rows'].iloc[current_row_index])):
            previous_data = getArrayData(current_row_index, i)
            for j in range(current_row_index + 1, total_rows):
                next_data = getRowData(j)
                if (abs(previous_data[0] - next_data[0]) < acceptable_range): #link on row to the next
    
                    path = findBestCombi(j)
    
                    # There is a valid path
                    if path and path['energy_consumption']:
                        curr_path = {
                            'energy_consumption': [previous_data[3]] + path['energy_consumption'],
                            'time_taken': [previous_data[2]] + path['time_taken'],
                            'starting_temp': previous_data[0],
                            'starting_humi': previous_data[1],
                            'ending_temp': path['ending_temp'],
                            'ending_humi': path['ending_humi'],
                            'path': [current_row_index] + path['path'] 
                        }
                        curr_path['factor'] = sum(curr_path['energy_consumption']) * energy_factor + sum(curr_path['time_taken']) * time_factor
                        if current_row_index in paths:
                            if comparePath(paths[current_row_index], curr_path):
                                paths[current_row_index] = curr_path
                        else:
                            paths[current_row_index] = curr_path
        # No valid paths to target temp and humi
        if current_row_index not in paths:
            
            paths[current_row_index] = {
                'energy_consumption': [], 
                'starting_temp': curr_temperature, 
                'starting_humi': curr_humidity, 
                'time_taken': [], 
                'factor': float('inf'), 
                'path': []}
        
        # return best path or empty path
        return paths[current_row_index]
    
    # TO get best path
    # find current temp and humi, then sort by 'comparisons'
    for i in range(total_rows):
        # data_temperature, data_humidity = getRowData(i)
        # print(i)
        if i not in paths:
            for j in range(len(aircon_status_result['rows'].iloc[i])):
                array_data = getArrayData(i, j)
                path = findBestCombi(i)
    
                # There is a valid path
                if path and path['energy_consumption']:
                    curr_path = {
                        'energy_consumption': [array_data[3]] + path['energy_consumption'],
                        'time_taken': [array_data[2]] + path['time_taken'],
                        'starting_temp': array_data[0],
                        'starting_humi': array_data[1],
                        'ending_temp': path['ending_temp'],
                        'ending_humi': path['ending_humi'],
                        'path': [i] + path['path'] 
                    }
                    curr_path['factor'] = sum(curr_path['energy_consumption']) * energy_factor + sum(curr_path['time_taken']) * time_factor
                    if i in paths:
                        if comparePath(paths[i], curr_path):
                            paths[i] = curr_path
                    else:
                        paths[i] = curr_path
                else:
                    paths[i] = {'energy_consumption': [], 'starting_temp': array_data[0], 'starting_humi': array_data[1], 'time_taken': [], 'factor': float('inf'), 'path': []}
    
    # Select the best alternative if no valid paths exist
    if not any(path['factor'] < float('inf') for path in paths.values()):
        print(f"Target temp {target_temp} not achievable, selecting next best value.")
        best_alternative = min(paths.values(), key=lambda p: p['factor'])
        stored_dictionary[f"Dictionary for target temp: {target_temp}"] = best_alternative
    else:
        stored_dictionary[f"Dictionary for target temp: {target_temp}"] = paths.copy()


end_time = time.time()
elapsed_time_minutes = (end_time - start_time) / 60
print(f"Processing completed in {elapsed_time_minutes:.2f} minutes")

Target temp 20 not achievable, selecting next best value.
Target temp 21 not achievable, selecting next best value.
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found
target found


In [24]:
#asssuming there's multiple key for target temp


current_temperature = 28
target_temp = 25
key_to_find = f"Dictionary for target temp: {target_temp}"




def expandPath(row_index):
    for i in range(1, aircon_units + 1):
        print("UNIT " + str(i) + ":", end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Status"].iloc[row_index], end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Fan_Status"].iloc[row_index], end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Set_Point"].iloc[row_index], end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Operation_Mode"].iloc[row_index], end=' ')
        print("")
        
def findClosestTemperature(current_temp, paths):
    """Find the index of the closest available temperature in the paths dictionary."""
    closest_temp_index = None
    smallest_difference = float('inf')
    
    for key, value in paths.items():
        if value['starting_temp'] and value['path']:  #ensure that paths[index] has both values in 'starting_temp' and 'path'
            difference = abs(value['starting_temp'] - current_temp)
            if difference < smallest_difference:
                smallest_difference = difference
                closest_temp_index = key   
    return closest_temp_index


# Check if either key exists in stored_dictionary
if key_to_find in stored_dictionary:
    stored_dict_key = stored_dictionary[key_to_find]


    filtered_paths = {
        key: value for key, value in stored_dict_key.items()
        if (abs(value['starting_temp'] - current_temperature) < acceptable_range)
    }
    
    if filtered_paths:
        # Find the path with the smallest factor
        smallest_factor_path = min(filtered_paths.keys(), key=lambda x: filtered_paths[x]['factor'])
        print(stored_dict_key[smallest_factor_path])
    
        for index, value in enumerate(stored_dict_key[smallest_factor_path]['path']):
            expandPath(value)
            print("For", end=" ")
            hours, remainder = divmod(stored_dict_key[smallest_factor_path]['time_taken'][index], 3600)
            minutes, seconds = divmod(remainder, 60)
            print(f"Hours: {hours}, Minutes: {minutes}, Seconds: {seconds}")
            print("")
    else:
        print("No paths found within the acceptable range.")
        print("Finding the closest temperature in the algorithm.")
    
        closest_temp_index = findClosestTemperature(current_temperature, paths)
        
        if closest_temp_index is not None:
            
            print("Closest temperature found at index:", closest_temp_index , "\n")
            print(paths[closest_temp_index])
            
            for index, value in enumerate(stored_dict_key[closest_temp_index]['path']):
                expandPath(value)
                print("For", end=" ")
                hours, remainder = divmod(stored_dict_key[closest_temp_index]['time_taken'][index], 3600)
                minutes, seconds = divmod(remainder, 60)
                print(f"Hours: {hours}, Minutes: {minutes}, Seconds: {seconds}")
                print("")
        else:
            print("No valid paths available, even for the closest temperature.")

else:
    print(f"{key_to_find} does not exist")





{'energy_consumption': [0.54, 0.0, 0.33, 0.0], 'time_taken': [275, 9, 9, 8], 'starting_temp': 27.26, 'starting_humi': 71.79, 'ending_temp': 25.34, 'ending_humi': 52.0, 'path': [1344, 1586, 2903, 3093], 'factor': 150.935}
UNIT 1: ON VERY LOW 24.0 COOL 
UNIT 2: ON VERY LOW 25.0 COOL 
UNIT 3: ON VERY LOW 22.0 COOL 
UNIT 4: ON VERY LOW 25.0 COOL 
UNIT 5: OFF OFF 23.0 COOL 
UNIT 6: OFF OFF 23.0 COOL 
UNIT 7: OFF OFF 23.0 COOL 
UNIT 8: OFF OFF 23.0 COOL 
For Hours: 0, Minutes: 4, Seconds: 35

UNIT 1: OFF OFF 22.8 DRY 
UNIT 2: OFF OFF 23.4 DRY 
UNIT 3: ON MID 23.0 COOL 
UNIT 4: OFF OFF 23.3 DRY 
UNIT 5: ON VERY LOW 23.5 COOL 
UNIT 6: ON VERY LOW 22.6 COOL 
UNIT 7: ON VERY LOW 23.5 COOL 
UNIT 8: OFF OFF 22.7 DRY 
For Hours: 0, Minutes: 0, Seconds: 9

UNIT 1: OFF OFF 20.0 COOL 
UNIT 2: ON LOW 23.4 COOL 
UNIT 3: ON MID 23.0 DRY 
UNIT 4: ON VERY LOW 22.7 DRY 
UNIT 5: ON HIGH 23.2 DRY 
UNIT 6: OFF OFF 23.4 COOL 
UNIT 7: ON MID 22.8 DRY 
UNIT 8: OFF OFF 22.7 COOL 
For Hours: 0, Minutes: 0, Seconds:

In [25]:
unique_keys = [key for key in stored_dictionary.keys() if key.startswith("Dictionary for target temp") or key.startswith("Best alternative for target temp")]
print(unique_keys)

['Dictionary for target temp: 20', 'Dictionary for target temp: 21', 'Dictionary for target temp: 22', 'Dictionary for target temp: 23', 'Dictionary for target temp: 24', 'Dictionary for target temp: 25', 'Dictionary for target temp: 26', 'Dictionary for target temp: 27', 'Dictionary for target temp: 28', 'Dictionary for target temp: 29']
